step 1 :Installing and importing necessary libraries

In [1]:
import cv2
import numpy as np
import os
import csv
import time
from datetime import datetime
from win32com.client import Dispatch
from keras.models import model_from_json
from keras.preprocessing import image

Step 2: Designing the GUI and Loading the Pretrained Model

In [ ]:
def speak(str1):
    speak = Dispatch("SAPI.SpVoice")
    speak.Speak(str1)

# Load the FaceNet model
def load_facenet_model(json_path, weights_path):
    with open(json_path, "r") as json_file:
        model_json = json_file.read()
    facenet_model = model_from_json(model_json)
    facenet_model.load_weights(weights_path)
    print("FaceNet model loaded successfully from JSON and weights.")
    return facenet_model

# Paths to FaceNet model files
facenet_json_path = r"C:\Users\MY\DSP1\db\facenet_model.json"
facenet_weights_path = r"C:\Users\MY\DSP1\db\facenet_model_weights.h5"

# Initialize FaceNet model
facenet_model = load_facenet_model(facenet_json_path, facenet_weights_path)

# Path to Haar Cascade XML for face detection
facedetect = cv2.CascadeClassifier(r"C:\Users\MY\DSP1\db\haarcascade_frontalface_default.xml")

# Load background image for the GUI
imgBackground = cv2.imread(r"C:\Users\MY\yolov8\roboflow screenshot\background.png")
COL_NAMES = ['NAME:', 'TIME']

Step 3 Real-Time Attendance System

In [ ]:
# Open the webcam
video = cv2.VideoCapture(0)

while True:
    ret, frame = video.read()
    if not ret:
        print("Error: Failed to capture image")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    detected_faces = facedetect.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in detected_faces:
        crop_image = frame[y:y + h, x:x + w, :]
        resized_img = cv2.resize(crop_image, (160, 160))  # Resize for FaceNet input
        img_array = image.img_to_array(resized_img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)

        # Generate embeddings using FaceNet
        embeddings = facenet_model.predict(img_array)

        # Match the embedding with stored embeddings
        output = "Recognized_Person"  # Replace with actual matching logic

        # Record timestamp
        ts = time.time()
        date = datetime.fromtimestamp(ts).strftime("%d-%m-%Y")
        timestamp = datetime.fromtimestamp(ts).strftime("%H:%M:%S")

        # Draw rectangle and name on the frame
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
        cv2.putText(frame, str(output), (x, y - 15), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        # Prepare attendance record
        attendance = [str(output), str(timestamp)]

    imgBackground[162:162 + 480, 55:55 + 640] = frame
    cv2.imshow("Frame", imgBackground)

    # Save attendance on key press
    k = cv2.waitKey(1)
    if k == ord('o'):  # Press 'o' to save attendance
        speak("Attendance Taken, Have a nice day!!")
        time.sleep(3)
        save_attendance_to_csv(attendance, date)

    if k == ord('q'):  # Press 'q' to quit
        break

# Release resources
video.release()
cv2.destroyAllWindows()

Step 4 Automatically Saving Attendance into CSV File

In [ ]:
def save_attendance_to_csv(attendance, date):
    filename = "Attendance/Attendance_" + date + ".csv"
    file_exists = os.path.isfile(filename)

    if file_exists:
        with open(filename, "a") as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(attendance)
    else:
        with open(filename, "w") as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(COL_NAMES)
            writer.writerow(attendance)
    print(f"Attendance saved to {filename}")

In [ ]:
import cv2
import numpy as np
import os
import csv
import pandas as pd
import time
from datetime import datetime
from win32com.client import Dispatch
import tensorflow as tf
from keras.models import load_model
from scipy.spatial.distance import cosine

def speak(str1):
    speak = Dispatch(("SAPI.SpVoice"))
    speak.Speak(str1)

# Load the FaceNet model
model_path = 'path_to_your_facenet_model.h5'  # Specify the path to the FaceNet model
facenet_model = load_model(model_path)

def preprocess_image(image):
    image = cv2.resize(image, (160, 160))  # Resize to 160x160
    image = np.expand_dims(image, axis=0)
    image = (image - 127.5) / 127.5  # Normalize the image
    return image

def get_face_embedding(image, model):
    preprocessed_image = preprocess_image(image)
    embedding = model.predict(preprocessed_image)
    return embedding

# Open the webcam
video = cv2.VideoCapture(0)
facedetect = cv2.CascadeClassifier(r"C:\Users\MY\DSP1\db\haarcascade_frontalface_default.xml")

# Load faces and names from CSV
faces = pd.read_csv(r"C:\users\my\DSP1\db\faces.csv", header=None).values  # Faces CSV should be embeddings
names = pd.read_csv(r"C:\users\my\DSP1\db\names.csv", header=None).values.flatten()  # Names CSV should be a single column of names

print(f"Faces length: {len(faces)}")
print(f"Names length: {len(names)}")

# Check if faces and names have the same length
if len(faces) == len(names):
    # Precompute the embeddings for the known faces
    known_face_embeddings = []
    for face in faces:
        known_face_embeddings.append(np.array(face))
    known_face_embeddings = np.array(known_face_embeddings)
else:
    print("Error: The number of faces and names do not match.")

imgBackground = cv2.imread(r"C:\Users\MY\yolov8\roboflow screenshot\background.png")

COL_NAMES = ['NAME:', 'TIME']
while True:
    ret, frame = video.read()
    if not ret:
        print("Error: Failed to capture image")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    detected_faces = facedetect.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in detected_faces:
        crop_image = frame[y:y + h, x:x + w, :]
        face_embedding = get_face_embedding(crop_image, facenet_model)

        # Compare the face embedding with known faces
        distances = []
        for known_embedding in known_face_embeddings:
            dist = cosine(face_embedding, known_embedding)  # Compute cosine similarity
            distances.append(dist)

        # Get the most similar face
        min_distance_index = np.argmin(distances)
        output = names[min_distance_index]  # The name of the recognized employee

        ts = time.time()
        date = datetime.fromtimestamp(ts).strftime("%d-%m-%Y")
        timestamp = datetime.fromtimestamp(ts).strftime("%H : %M : %S")
        exist = os.path.isfile("Attendance/Attendance_" + date + ".csv")
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 1)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (50, 50, 255), 2)
        cv2.rectangle(frame, (x, y - 40), (x + w, y), (50, 50, 255), -1)
        cv2.putText(frame, str(output), (x, y - 15), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 1)

        attendance = [str(output), str(timestamp)]

    imgBackground[162:162 + 480, 55:55 + 640] = frame
    cv2.imshow("Frame", imgBackground)

    k = cv2.waitKey(1)
    if k == ord('o'):
        speak("Attendance Taken, Have a nice day!!")
        time.sleep(3)
        if exist:
            with open("Attendance/Attendance_" + date + ".csv", "+a") as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow(attendance)
            csvfile.close()
        else:
            with open("Attendance/Attendance_" + date + ".csv", "+a") as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow(COL_NAMES)
                writer.writerow(attendance)
            csvfile.close()

    if k == ord('o'):
        break

# Release the webcam and close the window
video.release()
cv2.destroyAllWindows()

Step 1:

In [ ]:
import cv2
import numpy as np
import os
import csv
import time
from datetime import datetime
from win32com.client import Dispatch
from keras.models import model_from_json
from keras.preprocessing import image
import scipy.spatial

Step 2:

In [ ]:
def speak(str1):
    speak = Dispatch("SAPI.SpVoice")
    speak.Speak(str1)

# Load the FaceNet model
def load_facenet_model(json_path, weights_path):
    with open(json_path, "r") as json_file:
        model_json = json_file.read()
    facenet_model = model_from_json(model_json)
    facenet_model.load_weights(weights_path)
    print("FaceNet model loaded successfully from JSON and weights.")
    return facenet_model

# Paths to FaceNet model files
facenet_json_path = r"C:\Users\MY\DSP1\db\facenet_model.json"
facenet_weights_path = r"C:\Users\MY\DSP1\db\facenet_model_weights.h5"

# Initialize FaceNet model
facenet_model = load_facenet_model(facenet_json_path, facenet_weights_path)

# Path to Haar Cascade XML for face detection
facedetect = cv2.CascadeClassifier(r"C:\Users\MY\DSP1\db\haarcascade_frontalface_default.xml")

# Load background image for the GUI
imgBackground = cv2.imread(r"C:\Users\MY\yolov8\roboflow screenshot\background.png")
COL_NAMES = ['NAME:', 'TIME']

Step 3:

In [ ]:
# Paths to dataset images for known faces
known_faces_dir = r'C:\Users\MY\DSP1\db\train_augmented\'  
known_faces = ['LIM JING ROU', 'ANIS AQILAH', 'YU JIA ANN']  

# Store embeddings for known faces
known_embeddings = {}
for person in known_faces:
    img_path = os.path.join(known_faces_dir, person + '.jpg')
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    detected_faces = facedetect.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in detected_faces:
        crop_image = img[y:y + h, x:x + w]
        resized_img = cv2.resize(crop_image, (160, 160))  # Resize for FaceNet input
        img_array = image.img_to_array(resized_img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)
        embedding = facenet_model.predict(img_array)
        known_embeddings[person] = embedding

# Function to find the best match from known faces
def match_face(embedding):
    min_distance = float('inf')
    identified_person = "Unknown"
    for person, known_embedding in known_embeddings.items():
        distance = scipy.spatial.distance.euclidean(embedding, known_embedding)
        if distance < min_distance:
            min_distance = distance
            identified_person = person
    return identified_person

# Open the webcam
video = cv2.VideoCapture(0)

while True:
    ret, frame = video.read()
    if not ret:
        print("Error: Failed to capture image")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    detected_faces = facedetect.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in detected_faces:
        crop_image = frame[y:y + h, x:x + w, :]
        resized_img = cv2.resize(crop_image, (160, 160))  # Resize for FaceNet input
        img_array = image.img_to_array(resized_img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)

        # Generate embeddings using FaceNet
        embedding = facenet_model.predict(img_array)

        # Match the embedding with stored embeddings
        output = match_face(embedding)

        # Record timestamp
        ts = time.time()
        date = datetime.fromtimestamp(ts).strftime("%d-%m-%Y")
        timestamp = datetime.fromtimestamp(ts).strftime("%H:%M:%S")

        # Draw rectangle and name on the frame
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
        cv2.putText(frame, str(output), (x, y - 15), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        # Prepare attendance record
        attendance = [str(output), str(timestamp)]

    imgBackground[162:162 + 480, 55:55 + 640] = frame
    cv2.imshow("Frame", imgBackground)
    
    # Save attendance on key press
    k = cv2.waitKey(1)
    if k == ord('o'):  # Press 'o' to save attendance
        speak("Attendance Taken, Have a nice day!!")
        time.sleep(3)
        save_attendance_to_csv(attendance, date)

    if k == ord('o'):  # Press 'o' to quit
        break

# Release resources
video.release()
cv2.destroyAllWindows()

Step 4:

In [ ]:
# Function to save attendance to CSV
def save_attendance_to_csv(attendance, date):
    filename = "Attendance/Attendance_" + date + ".csv"
    file_exists = os.path.isfile(filename)

    if file_exists:
        with open(filename, "a") as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(attendance)
    else:
        with open(filename, "w") as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(COL_NAMES)
            writer.writerow(attendance)
    print(f"Attendance saved to {filename}")